# Modeling 3 Enzymatic Pathways with the Rheostat
In this notebook, we will take a look at a couple different enzymatic pathways and try to understand which one we would prefer to model.

In [1]:
#Using Bioscrape: Basic Imports

#A Model is a CRN with some bells and whistles
from bioscrape.types import Model

#py_simulate_model is a helper function that takes care of may details for you
from bioscrape.simulator import py_simulate_model

#For arrays and plotting
import numpy as np
import pylab as plt

# Import good plotting packages 
import bokeh.io
import bokeh.plotting
#import bokeh_catplot


bokeh.io.output_notebook()

Loading BokehJS ...

Now, we will introduce a half-way point (with fructose-1,6-phosphate) and attempt to model the three different ways that enzyme substrate binding can occur. The half-way point is necessary so we can get the free phosphate concentrations that come from the Non-ATP generating pathway.

# Take a quick look at the simplified model

**Combined**
$$\text{2 ATP + Glucose} \leftrightarrow^{E1} \text{2 ADP + F16P}$$
**Non-ATP Generating**
$$\text{2 ADP + F16P} \leftrightarrow^{E2} \text{2 ATP + Isobutanol}$$
**ATP Generating**
$$\text{3 ADP + Pi + F16P} \leftrightarrow^{E3} \text{3 ATP + Isobutanol}$$
**Spontaneous Hydrolysis / Degradation**
$$\text{ATP} \rightarrow \text{ADP + Pi}$$



In [2]:
species = ['atp', 'glucose', 'adp', 'pi', 'f16p', 'isobutanol']

parameters = [('kf', 1), ('kr', 0.1), ('delta', 10),('k_glu', 100)]

rxn1 = (['atp', 'atp', 'glucose'], ['adp', 'adp', 'f16p'], 'massaction', {'k': 'kf'})
rxn2 = (['adp', 'adp', 'f16p'], ['atp', 'atp', 'isobutanol'], 'massaction', {'k': 'kf'})
rxn3 = (['adp', 'adp', 'adp', 'pi', 'f16p'], ['atp', 'atp', 'atp', 'isobutanol'], 'massaction', {'k': 'kf'})
rxn_glu = ([], ['glucose'], 'massaction', {'k':'k_glu'})
rxn_atp = ([], ['atp'], 'massaction', {'k':'k_glu'})

rxn_d_atp = (['atp'], ['adp', 'pi'], 'massaction', {'k': 'delta'})


reactions = [rxn1, rxn2, rxn3, rxn_d_atp, rxn_glu]

x_0 = {
    'glucose': 100,
    'atp':200
}

M = Model(species = species, reactions = reactions, parameters = parameters,
         initial_condition_dict = x_0)
timepoints = np.linspace(0,1000,1000)
df_results = py_simulate_model(timepoints, Model = M, stochastic = False)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: The following species are uninitialized and their value has been defaulted to 0: adp, pi, f16p, isobutanol, 
/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
odeint failed with mxstep=500...

In [3]:
# First plot glucose and isobutanol
p = bokeh.plotting.figure(width = 300, height = 250, 
                         x_axis_label = 'time',
                         y_axis_label = 'concentration')
p.line(timepoints, df_results['glucose'], color = 'green', legend_label = 'glucose')
p.line(timepoints, df_results['isobutanol'], color = 'orange', legend_label = 'isobutanol')
p.line(timepoints, df_results['f16p'], color = 'purple', legend_label = 'f16p')


p.legend.location = 'center_right'

# Plot ATP and ADP, NADPH, NADP+
p2 = bokeh.plotting.figure(width = 300, height = 250,
                           x_axis_label = 'time',
                         y_axis_label = 'concentration')
p2.line(timepoints, df_results['atp'], color = 'red', legend_label = 'atp')
p2.line(timepoints, df_results['pi'], color = 'blue', legend_label = 'pi')
p2.line(timepoints, df_results['adp'], color = 'green', legend_label = 'adp')
#p2.line(timepoints, df_results['atp_extra'], color = 'purple', legend_label = 'atp_extra')
p2.legend.location = 'center_right'


# Show plots
bokeh.io.show(p)
bokeh.io.show(p2)

We see one atp regeneration cycle. Most likely due to the fact there is no more f16p remaining, reactions 2 or 3 cannot proceed.

In [4]:

atp_vals = list(df_results['atp'].values)
atp_vals.remove(max(atp_vals))
max_atp = max(atp_vals)
time = atp_vals.index(max_atp)
timepoints[time]

0.0

## Try modeling the 1 step model with massaction

$$\text{Fuel + Substrate + Enzyme} \leftrightarrow^1 \text{Fuel:Substrate:Enzyme} \rightarrow^2 \text{Waste:Product:Enzyme} \leftrightarrow^3 \text{Waste + Product + Enzyme}$$

**Combined** <br>
$$\text{2 ATP + Glucose + Enzyme1} \leftrightarrow^1 \text{2ATP:Glucose:Enzyme1} \rightarrow^2 \text{2ADP:F16P:Enzyme1} \leftrightarrow^3 \text{2 ADP + F16P + Enzyme1}$$ <br>

**Non-ATP Generating Pathway** <br>
$$\text{F16P + 2 ADP + Enzyme2} \leftrightarrow^4 \text{2ADP:F16P:Enzyme2} \rightarrow^5 \text{2 ATP:Isobutanol:Enzyme2} \leftrightarrow^6 \text{2 ATP + Isobutanol + Enzyme2}$$ <br>

**ATP Generating Pathway** <br>
$$\text{3 ADP + Pi + F16P + Enzyme3} \leftrightarrow^7 \text{3ADPPi:F16P:Enzyme3} \rightarrow^8 \text{3ATP:Isobutanol:Enzyme3} \leftrightarrow^9 \text{3 ATP + Isobutanol + Enzyme3}$$ <br>

**ATP Degradation** <br>
$$\text{ATP} \rightarrow \text{ADP + Pi}$$


In [5]:
species = ['atp', 'glucose', 'e1', '2atp:glucose:e1', '2adp:f16p:e1', 'adp', 'pi', 'f16p',
           'e2', '2adp:f16p:e2','2atp:isobutanol:e2', 
           '3adppi:f16p:e3', 'e3' , '3atp:isobutanol:e3', 'isobutanol']


parameters = [('kf', 10), ('kr', 0.1), ('delta', 10)]



# reaction 1
rxn1_f = (['atp', 'atp', 'glucose', 'e1'], ['2atp:glucose:e1'],
           'massaction',  {'k': 'kf'})
                            
rxn1_r = ( ['2atp:glucose:e1'],['atp', 'atp', 'glucose', 'e1'], 
            'massaction', {'k': 'kr'})

# reaction 2
rxn2_f = ( ['2atp:glucose:e1'],['2adp:f16p:e1'], 
           'massaction', {'k': 'kf'})

# reaction 3
rxn3_f =  ( ['2adp:f16p:e1'], ['adp', 'adp', 'f16p', 'e1'], 
           'massaction', {'k': 'kf'})
 
rxn3_r =  (['adp', 'adp', 'f16p', 'e1'], ['2adp:f16p:e1'],
           'massaction', {'k': 'kr'})

# reaction 4
rxn4_f =  (['f16p', 'adp', 'adp', 'e2'], ['2adp:f16p:e2'],
           'massaction', {'k': 'kf'})

rxn4_r =  ( ['2adp:f16p:e2'], ['f16p', 'adp', 'adp', 'e2'],
           'massaction', {'k': 'kr'})

# reaction 5 
rxn5_f =  (['2adp:f16p:e2'], ['2atp:isobutanol:e2'],
           'massaction', {'k': 'kf'})


# reaction 6
rxn6_f =  (['2atp:isobutanol:e2'], ['atp', 'atp', 'isobutanol', 'e2'],
           'massaction', {'k': 'kf'})

rxn6_r =  (['atp', 'atp', 'isobutanol', 'e2'], ['2atp:isobutanol:e2'],
           'massaction', {'k': 'kr'})
    

# reaction 7
rxn7_f = ( ['adp', 'adp', 'adp', 'pi', 'f16p', 'e3'],['3adppi:f16p:e3'], 
              'massaction', {'k': 'kf'})

rxn7_r = ( ['3adppi:f16p:e3'],['adp', 'adp', 'adp', 'pi', 'f16p', 'e3'], 
              'massaction', {'k': 'kr'})

# reaction 8
rxn8_f = ( ['3adppi:f16p:e3'],['3atp:isobutanol:e3'], 
              'massaction', {'k': 'kf'})

# reaction 9
rxn9_f = ( ['3atp:isobutanol:e3'],['atp', 'atp','atp',  'isobutanol', 'e3'], 
              'massaction', {'k': 'kf'})

rxn9_r = ( ['atp', 'atp', 'atp', 'isobutanol', 'e3'],['3atp:isobutanol:e3'], 
              'massaction', {'k': 'kr'})

# ATP DEGRADATION
rxn_d_atp = (["atp"], ['adp', 'pi'], "massaction", {"k":"delta"})



reactions = [rxn1_f, rxn1_r, rxn2_f, rxn3_f, rxn3_r,rxn4_f,
             rxn4_r, rxn5_f, rxn6_f,  rxn6_r, rxn7_f,
             rxn7_r, rxn8_f, rxn9_f, rxn9_r, rxn_d_atp]



# Initial Conditions             
x0 = {
    "glucose":100,
    "atp":200, 
    'e1': 200, 
    'e2':200,
    'e3':200
}

rules = []

M = Model(species = species, reactions = reactions, parameters = parameters, rules = rules, initial_condition_dict = x0)
timepoints = np.linspace(0,1000,1000)
df_results = py_simulate_model(timepoints, Model = M, stochastic = False)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:88: UserWarning: The following species are uninitialized and their value has been defaulted to 0: 2atp:glucose:e1, 2adp:f16p:e1, adp, pi, f16p, 2adp:f16p:e2, 2atp:isobutanol:e2, 3adppi:f16p:e3, 3atp:isobutanol:e3, isobutanol, 
odeint failed with mxstep=500...

In [10]:
# First plot glucose and isobutanol
p = bokeh.plotting.figure(width = 300, height = 250, 
                         x_axis_label = 'time',
                         y_axis_label = 'concentration')
p.line(timepoints, df_results['glucose'], color = 'green', legend_label = 'glucose')
p.line(timepoints, df_results['isobutanol'], color = 'orange', legend_label = 'isobutanol')
p.line(timepoints, df_results['f16p'], color = 'purple', legend_label = 'f16p')
p.legend.location = 'center_right'

# Plot ATP and ADP, NADPH, NADP+
p2 = bokeh.plotting.figure(width = 300, height = 250,
                           x_axis_label = 'time',
                         y_axis_label = 'concentration')
p2.line(timepoints, df_results['atp'], color = 'red', legend_label = 'atp')
p2.line(timepoints, df_results['pi'], color = 'blue', legend_label = 'pi')
p2.line(timepoints, df_results['adp'], color = 'green', legend_label = 'adp')
p2.legend.location = 'center_right'


# Show plots

bokeh.io.show(p)
bokeh.io.show(p2)

In [17]:
df_results

,atp,glucose,e1,2atp:glucose:e1,2adp:f16p:e1,adp,pi,f16p,e2,2adp:f16p:e2,2atp:isobutanol:e2,3adppi:f16p:e3,e3,3atp:isobutanol:e3,isobutanol,time
0,200.000000,1.000000e+02,200.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,200.000000,0.000000e+00,0.000000,0.000000e+00,200.000000,0.000000,0.000000,0.000000
1,1.082672,2.088190e-07,199.949056,4.500105e-03,4.644418e-02,4.277779,4.277752,1.337777e-05,161.050417,1.107621e-01,38.838821,1.204692e-01,161.120502,38.759029,22.119961,1.001001
2,0.872617,1.165461e-11,199.999996,2.022564e-07,4.152319e-06,13.939703,13.939703,1.473737e-10,162.962511,7.676967e-06,37.037482,3.391413e-05,162.962436,37.037530,25.924942,2.002002
3,0.738130,5.129946e-16,200.000000,7.458615e-12,2.395635e-10,21.956904,21.956904,1.038727e-13,164.539007,4.263813e-10,35.460993,3.292053e-09,164.539007,35.460993,29.078013,3.003003
4,0.644037,4.800257e-19,200.000000,6.127543e-15,-3.066539e-12,28.849746,28.849746,1.153135e-13,165.898757,-2.499623e-10,34.101243,2.430314e-10,165.898757,34.101243,31.797513,4.004004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.000004,-1.408279e-40,200.000000,-2.114669e-42,-1.996732e-26,199.996120,199.996120,7.868348e-34,199.999225,1.961131e-29,0.000775,3.896446e-27,199.999225,0.000775,99.998449,995.995996
996,0.000004,-1.239446e-40,200.000000,-2.089600e-42,-1.153341e-26,199.996158,199.996158,9.394412e-34,199.999232,2.659714e-29,0.000768,5.302359e-27,199.999232,0.000768,99.998465,996.996997
997,0.000004,-1.066191e-40,200.000000,-2.034233e-42,-8.336590e-28,199.996196,199.996196,1.072242e-33,199.999240,3.344628e-29,0.000760,6.682792e-27,199.999240,0.000760,99.998480,997.997998
998,0.000004,-8.911028e-41,200.000000,-1.946206e-42,1.239221e-26,199.996234,199.996234,1.174756e-33,199.999248,3.984293e-29,0.000752,7.974728e-27,199.999248,0.000752,99.998495,998.998999


We see no atp regeneration.

## Try modeling the three-step model version 1


$$\text{Substrate + Enzyme} \leftrightarrow^1 \text{Substrate:Enzyme}$$ <br>
$$\text{Fuel + Substrate:Enzyme} \leftrightarrow^2 \text{Fuel:Substrate:Enzyme} \rightarrow^3 \text{Waste:Product:Enzyme} \leftrightarrow^4 \text{Waste + Product:Enzyme}$$ <br>
$$\text{Product:Enzyme} \leftrightarrow^5 \text{Product + Enzyme}$$ <br>

**Common**
$$\text{Glucose + Enzyme1} \leftrightarrow^1 \text{Glucose:Enzyme1}$$ <br>
$$\text{2 ATP + Glucose:Enzyme1} \leftrightarrow^2 \text{2ATP:Glucose:Enzyme1} \rightarrow^3 \text{2adp:f16p:Enzyme1} \leftrightarrow^4 \text{2ADP + F16P:Enzyme1}$$ <br>
$$\text{F16P:Enzyme1} \leftrightarrow^5 \text{F16P + Enzyme1}$$ <br>

**Non-ATP Generating Pathway**
$$\text{F16P + Enzyme2} \leftrightarrow^6 \text{F16P:Enzyme2}$$ <br>
$$\text{2 ADP + F16P:Enzyme2} \leftrightarrow^7 \text{2adp:F16P:Enzyme2} \rightarrow^8 \text{2ATP:Isobutanol:Enzyme2} \leftrightarrow^9 \text{2 ATP + Isobutanol:Enzyme2}$$ <br>
$$\text{Isobutanol:Enzyme2} \leftrightarrow^{10} \text{Isobutanol + Enzyme2}$$ <br>


**ATP Generating Pathway**
$$\text{F16P + Enzyme3} \leftrightarrow^{11} \text{F16P:Enzyme3}$$ <br>
$$\text{3 ADP + Pi + F16P:Enzyme3} \leftrightarrow^{12} \text{3adppi:F16P:Enzyme3} \rightarrow^{13} \text{3ATP:Isobutanol:Enzyme3} \leftrightarrow^{14} \text{3 ATP + Isobutanol:Enzyme3}$$ <br>
$$\text{Isobutanol:Enzyme3} \leftrightarrow^{15} \text{Isobutanol + Enzyme3}$$ <br>


**ATP Degradation** <br>
$$\text{ATP} \rightarrow \text{ADP + Pi}$$

In [2]:
species = ['glucose', 'e1', 'atp', 'glucose:e1', '2atp:glucose:e1', '2adp:f16p:e1', 'adp', 'pi', 'f16p:e1', 'f16p',
          'e2', 'f16p:e2', '2adp:f16p:e2', '2atp:isobutanol:e2', 'isobutanol:e2',
          'e3', 'f16p:e3', '3adppi:f16p:e3', '3atp:isobutanol:e3', 'isobutanol:e3', 'isobutanol']

parameters = [('k_f', 10), ('k_r', 0.1), ('delta', 10)]



rxn1_f = (['glucose', 'e1'], ['glucose:e1'], 'massaction', {'k': 'k_f'})
rxn1_r = (['glucose:e1'], ['glucose', 'e1'], 'massaction', {'k': 'k_r'})

rxn2_f = (['atp', 'atp', 'glucose:e1'], ['2atp:glucose:e1'], 'massaction', {'k': 'k_f'})
rxn2_r = (['2atp:glucose:e1'], ['atp', 'atp', 'glucose:e1'], 'massaction', {'k': 'k_r'})

rxn3_f = (['2atp:glucose:e1'], ['2adp:f16p:e1'], 'massaction', {'k': 'k_f'})

rxn4_f = (['2adp:f16p:e1'], ['adp', 'adp', 'f16p:e1'], 'massaction', {'k': 'k_f'})
rxn4_r = (['adp', 'adp', 'f16p:e1'], ['2adp2pi:f16p:e1'], 'massaction', {'k': 'k_r'})

rxn5_f = (['f16p:e1'], ['f16p', 'e1'], 'massaction', {'k': 'k_f'})

rxn6_f = (['f16p', 'e2'], ['f16p:e2'], 'massaction', {'k': 'k_f'})
rxn6_r = (['f16p:e2'], ['f16p', 'e2'], 'massaction', {'k': 'k_r'})

rxn7_f = (['adp', 'adp','f16p:e2'], ['2adp:f16p:e2'], 'massaction', {'k': 'k_f'})
rxn7_r = (['2adp2pi:f16p:e2'], ['adp', 'adp', 'pi', 'pi', 'f16p:e2'], 'massaction', {'k': 'k_r'})

rxn8_f = (['2adp:f16p:e2'], ['2atp:isobutanol:e2'], 'massaction', {'k': 'k_f'})

rxn9_f = (['2atp:isobutanol:e2'], ['atp', 'atp', 'isobutanol:e2'], 'massaction', {'k': 'k_f'})
rxn9_r = (['atp', 'atp', 'isobutanol:e2'], ['2atp:isobutanol:e2'], 'massaction', {'k': 'k_r'})

rxn10_f = (['isobutanol:e2'], ['isobutanol', 'e2'], 'massaction', {'k': 'k_f'})
rxn10_r = (['isobutanol', 'e2'], ['isobutanol:e2'], 'massaction', {'k': 'k_r'})

rxn11_f = (['f16p', 'e3'], ['f16p:e3'], 'massaction', {'k': 'k_f'})
rxn11_r = ( ['f16p:e3'], ['f16p', 'e3'], 'massaction', {'k': 'k_r'})

rxn12_f = (['adp', 'adp', 'adp', 'pi', 'f16p:e3'], ['3adppi:f16p:e3'], 'massaction', {'k': 'k_f'})
rxn12_r = (['3adppi:f16p:e3'], ['adp', 'adp', 'adp', 'pi', 'f16p:e3'], 'massaction', {'k': 'k_r'})

rxn13_f = (['3adppi:f16p:e3'], ['3atp:isobutanol:e3'], 'massaction', {'k': 'k_f'})

rxn14_f = (['3atp:isobutanol:e3'], ['atp', 'atp', 'atp', 'isobutanol:e3'], 'massaction', {'k': 'k_f'})
rxn14_r = ( ['atp', 'atp', 'atp', 'isobutanol:e3'], ['3atp:isobutanol:e3'], 'massaction', {'k': 'k_r'})

rxn15_f = (['isobutanol:e3'], ['isobutanol', 'e3'], 'massaction', {'k': 'k_f'})
rxn15_r = (['isobutanol', 'e3'], ['isobutanol:e3'], 'massaction', {'k': 'k_r'})

rxn_d_atp = (["atp"], ['adp', 'pi'], "massaction", {"k":"delta"})



reactions = [rxn1_f, rxn1_r, rxn2_f, rxn2_r, rxn3_f, rxn4_f, rxn4_r, rxn5_f, rxn6_f, rxn6_r, rxn7_f, rxn7_r,
            rxn8_f, rxn9_f, rxn9_r, rxn10_f, rxn10_r, rxn11_f, rxn11_r, rxn12_f, rxn12_r, rxn13_f,
            rxn14_f, rxn14_r, rxn15_f, rxn15_r, rxn_d_atp]

x_0 = {
    'glucose': 100,
    'atp':200,
    'e1':200,
    'e2':200,
    'e3':200
}

M = Model(species = species, reactions = reactions, parameters = parameters,
         initial_condition_dict = x_0)
timepoints = np.linspace(0,10,1000)
df_results = py_simulate_model(timepoints, Model = M, stochastic = False)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:67: UserWarning: The following species are uninitialized and their value has been defaulted to 0: glucose:e1, 2atp:glucose:e1, 2adp:f16p:e1, adp, pi, f16p:e1, f16p, f16p:e2, 2adp:f16p:e2, 2atp:isobutanol:e2, isobutanol:e2, f16p:e3, 3adppi:f16p:e3, 3atp:isobutanol:e3, isobutanol:e3, isobutanol, 2adp2pi:f16p:e1, 2adp2pi:f16p:e2, 


In [8]:
# First plot glucose and isobutanol
p = bokeh.plotting.figure(width = 300, height = 250, 
                         x_axis_label = 'time',
                         y_axis_label = 'concentration')
p.line(timepoints, df_results['glucose'], color = 'green', legend_label = 'glucose')
p.line(timepoints, df_results['isobutanol'], color = 'orange', legend_label = 'isobutanol')
p.line(timepoints, df_results['f16p'], color = 'purple', legend_label = 'f16p')


p.legend.location = 'center_right'

# Plot ATP and ADP, NADPH, NADP+
p2 = bokeh.plotting.figure(width = 300, height = 250,
                           x_axis_label = 'time',
                         y_axis_label = 'concentration')
p2.line(timepoints, df_results['atp'], color = 'red', legend_label = 'atp')
p2.line(timepoints, df_results['pi'], color = 'blue', legend_label = 'pi')
p2.line(timepoints, df_results['adp'], color = 'green', legend_label = 'adp')
p2.legend.location = 'center_right'


# Show plots
bokeh.io.show(p)
bokeh.io.show(p2)

We see one atp regeneration cycle. The level of isobutanol is strangely low.

## Try modeling three-step model version 2

$$\text{Enzyme + Fuel} \leftrightarrow^1 \text{Enzyme:Fuel}$$ <br>
$$\text{Substrate + Enzyme:Fuel} \leftrightarrow^2 \text{Fuel:Substrate:Enzyme} \rightarrow^3 \text{Waste:Product:Enzyme} \leftrightarrow^4 \text{Waste:Enzyme + Product}$$ <br>
$$\text{Waste:Enzyme}\leftrightarrow^5 \text{Waste + Enzyme}$$ <br>

**Combined**
$$\text{Enzyme1 + 2 ATP} \leftrightarrow^1 \text{Enzyme1:2ATP}$$ <br>
$$\text{Glucose + Enzyme1:2ATP} \leftrightarrow^2 \text{2ATP:Glucose:Enzyme1} \rightarrow^3 \text{2ADP:F16P:Enzyme1} \leftrightarrow^4 \text{2ADP:Enzyme1 + F16P}$$ <br>
$$\text{2ADP:Enzyme1}\leftrightarrow^5 \text{2 ADP + Enzyme1}$$ <br>

**Non-ATP Generating Pathway**
$$\text{Enzyme2 + 2 ADP} \leftrightarrow^6 \text{Enzyme2:2ADP}$$ <br>
$$\text{F16P + Enzyme2:2ADP} \leftrightarrow^7 \text{2ADP:F16P:Enzyme2} \rightarrow^8 \text{2ATP:Isobutanol:Enzyme2} \leftrightarrow^9 \text{2ATP:Enzyme2 + Isobutanol}$$ <br>
$$\text{2ATP:Enzyme2}\leftrightarrow^{10} \text{2 ATP + Enzyme2}$$ <br>

**ATP Generating Pathway**
$$\text{Enzyme3 + 3 ADP + Pi} \leftrightarrow^{11} \text{Enzyme3:3ADPPi}$$ <br>
$$\text{F16P + Enzyme3:3ADPPi} \leftrightarrow^{12} \text{3ADPPi:F16P:Enzyme3} \rightarrow^{13} \text{3ATP:F16P:Enzyme3} \leftrightarrow^{14} \text{3ATP:Enzyme3 + F16P}$$ <br>
$$\text{3ATP:Enzyme3}\leftrightarrow^{15} \text{3 ATP + Enzyme3}$$ <br>



In [11]:
species = ['e1', 'atp', 'e1:2atp', '2atp:glucose:e1', '2adp:f16p:e1', '2adp:e1', 'f16p', '2adp:e1', 'adp', 'pi',
          'e2', 'e2:2adp', '2adp:f16p:e2', '2atp:isobutanol:e2', '2atp:e2', 'isobutanol',
          'e3', 'e2:3adppi', '3adppi:f16p:e3', '3atp:f16p:e3', '3atp:e3']

parameters = [('kf', 10), ('kr', 0.1), ('delta', 10)]

rxn1f = (['e1', 'atp', 'atp'], ['e1:2atp'], 'massaction', {'k': 'kf'})
rxn1r = (['e1:2atp'], ['e1', 'atp', 'atp'], 'massaction', {'k': 'kf'})

rxn2f = (['glucose', 'e1:2atp'], ['2atp:glucose:e1'], 'massaction', {'k': 'kf'})
rxn2r = (['2atp:glucose:e1'], ['glucose', 'e1:2atp'], 'massaction', {'k': 'kr'})

rxn3f = (['2atp:glucose:e1'], ['2adp:f16p:e1'], 'massaction', {'k': 'kf'})

rxn4f = (['2adp:f16p:e1'], ['2adp:e1', 'f16p'], 'massaction', {'k': 'kf'})
rxn4r = (['2adp:e1', 'f16p'], ['2adp:f16p:e1'], 'massaction', {'k': 'kr'})

rxn5f = (['2adp:e1'], ['adp', 'adp', 'e1'], 'massaction', {'k': 'kf'})
rxn5r = (['adp', 'adp', 'e1'], ['2adp:e1'], 'massaction', {'k': 'kr'})

rxn6f = (['e2', 'adp', 'adp'], ['e2:2adp'], 'massaction', {'k': 'kf'})
rxn6r = (['e2:2adp'], ['e2', 'adp', 'adp'], 'massaction', {'k': 'kf'})

rxn7f = (['f16p', 'e2:2adp'], ['2adp:f16p:e2'], 'massaction', {'k': 'kf'})
rxn7r = (['2adp:f16p:e2'], ['f16p', 'e2:2adp'], 'massaction', {'k': 'kf'})

rxn8f = (['2adp:f16p:e2'], ['2atp:isobutanol:e2'], 'massaction', {'k': 'kf'})

rxn9f = (['2atp:isobutanol:e2'], ['2atp:e2', 'isobutanol'], 'massaction', {'k': 'kf'})
rxn9r = (['2atp:e2', 'isobutanol'], ['2atp:isobutanol:e2'], 'massaction', {'k': 'kr'})

rxn10f = (['2atp:e2'], ['atp', 'atp', 'e2'], 'massaction', {'k': 'kf'})
rxn10r = (['atp', 'atp', 'e2'], ['2atp:e2'], 'massaction', {'k': 'kr'})

rxn11f = (['e3', 'adp', 'adp', 'adp', 'pi'], ['e3:3adppi'], 'massaction', {'k': 'kf'})
rxn11r = (['e3:3adppi'], ['e3', 'adp', 'adp', 'adp', 'pi'], 'massaction', {'k': 'kr'})

rxn12f = (['f16p', 'e3:3adppi'], ['3adppi:f16p:e3'], 'massaction', {'k': 'kf'})
rxn12r = (['3adppi:f16p:e3'], ['f16p', 'e3:3adppi'], 'massaction', {'k': 'kr'})

rxn13f = (['3adppi:f16p:e3'], ['3atp:f16p:e3'], 'massaction', {'k': 'kf'})

rxn14f = (['3atp:f16p:e3'], ['3atp:e3', 'f16p'], 'massaction', {'k': 'kf'})
rxn14r = (['3atp:e3', 'f16p'], ['3atp:f16p:e3'], 'massaction', {'k': 'kr'})

rxn15f = (['3atp:e3'], ['atp', 'atp', 'atp', 'e3'], 'massaction', {'k': 'kf'})
rxn15r = (['atp', 'atp', 'atp', 'e3'], ['3atp:e3'], 'massaction', {'k': 'kr'})

rxn_d_atp = (["atp"], ['adp', 'pi'], "massaction", {"k":"delta"})

reactions = [rxn1f, rxn1r, rxn2f, rxn2r, rxn3f, rxn4f, rxn4r, rxn5f, rxn5r, rxn6f, rxn6r, rxn7f, rxn7r,
            rxn8f, rxn9f, rxn9r, rxn10f, rxn10r, rxn11f, rxn11r, rxn12f, rxn12r, rxn13f, rxn14f, rxn14r,
            rxn15f, rxn15r, rxn_d_atp]

x_0 = {
    'glucose': 1000,
    'atp':100,
    'e1':10,
    'e2':10,
    'e3':10
}

M = Model(species = species, reactions = reactions, parameters = parameters,
         initial_condition_dict = x_0)
timepoints = np.linspace(0,100,1000)
df_results = py_simulate_model(timepoints, Model = M, stochastic = False)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:64: UserWarning: The following species are uninitialized and their value has been defaulted to 0: e1:2atp, 2atp:glucose:e1, 2adp:f16p:e1, 2adp:e1, f16p, adp, pi, e2:2adp, 2adp:f16p:e2, 2atp:isobutanol:e2, 2atp:e2, isobutanol, e2:3adppi, 3adppi:f16p:e3, 3atp:f16p:e3, 3atp:e3, e3:3adppi, 
/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
odeint failed with mxstep=500...

In [12]:
# First plot glucose and isobutanol
p = bokeh.plotting.figure(width = 300, height = 250, 
                         x_axis_label = 'time',
                         y_axis_label = 'concentration')
p.line(timepoints, df_results['glucose'], color = 'green', legend_label = 'glucose')
p.line(timepoints, df_results['isobutanol'], color = 'orange', legend_label = 'isobutanol')
p.line(timepoints, df_results['f16p'], color = 'purple', legend_label = 'f16p')
p.legend.location = 'center_right'

# Plot ATP and ADP, NADPH, NADP+
p2 = bokeh.plotting.figure(width = 300, height = 250,
                           x_axis_label = 'time',
                         y_axis_label = 'concentration')
p2.line(timepoints, df_results['atp'], color = 'red', legend_label = 'atp')
p2.line(timepoints, df_results['pi'], color = 'blue', legend_label = 'pi')
p2.line(timepoints, df_results['adp'], color = 'green', legend_label = 'adp')
p2.legend.location = 'center_right'


# Show plots
bokeh.io.show(p)
bokeh.io.show(p2)


Why is Pi still being created?

## Conclusions/Future Directions

- It seems the 3-step model version 1 gets me closest to the results I desire. 
    - Conceptually, it's slightly interesting as to why these different mechanisms are giving me different dynamics
    - Barring any coding errors, this is something to look further into
- The prominent issue with my model for the rheostat is that I get regeneration of only 1 ATP cycle. 
- Attempt to use bioCRNpyler and see how the results compare

In [14]:
%reload_ext watermark
%watermark -v -p numpy,bokeh,jupyterlab,biocircuits

CPython 3.7.5
IPython 7.13.0

numpy 1.18.1
bokeh 2.0.1
jupyterlab 1.2.4
biocircuits 0.0.15
